In [1]:
using Pkg
Pkg.activate(".")
using Flux
using SignalAnalysis
using WAV
using Plots
using MFCC
using DSP
using JLD2

DATA = "datasets/audioMNIST"
PREPROCESSED = "preprocessed/audioMNIST"
RESAMPLED = "$PREPROCESSED/resampled"
TRAINFILE = "$PREPROCESSED/train.jld2"
NFFT=512
NOVERLAP=128


  Activating project at `~/julia-ml`


128

In [2]:

spec = Spectrogram(nfft=NFFT, noverlap=NOVERLAP, window=hamming)

# spec = spectrogram(sample[:, 1], NFFT, NOVERLAP; fs, window=hanning)
# @info mfcc(sample, fs)

# sample[1:3:end, :]

responsetype = Lowpass(8000; fs=48000)
designmethod = Butterworth(4)
dofilt(x) = filt(digitalfilter(responsetype, designmethod), x)


for sname in Base.Filesystem.readdir(DATA; join=true)
    for fname in Base.Filesystem.readdir(sname)
        path = "$(sname)/$(fname)"
        sample, fs = wavread(path)
        resampled = dofilt(sample[1:3:end, 1])
        resampled = resampled ./ maximum(abs.(resampled))
        resampled = vcat(resampled, .01*randn(16000 - size(resampled, 1)))
        wavwrite(resampled, "$RESAMPLED/$fname"; Fs=fs/3)
    end
end


In [ ]:

result = zeros(98, 13, 60, 50, 10) # TODO don't hardcode 50 samples per digit

count = 0
for (i, fname) in enumerate(Base.Filesystem.readdir(RESAMPLED))
    path = "$(RESAMPLED)/$(fname)"
    sample, fs = wavread(path)

    count += 1

    
    digit = parse(Int, fname[1])
    sampleidx = parse(Int, fname[6:(end-4)]) 
    speakeridx = parse(Int, fname[3:4])


    m = mfcc(sample)[1]
    # display(maximum(abs.(sample)))
    # display(mfcc(sample))

    result[:, :, speakeridx, sampleidx+1, digit+1] = m

    # result[:, sampleidx+1, digit+1] = vcat(sample, zeros(16000 - size(sample, 1)))
end


# result = result ./ maximum(result)

jldsave("$TRAINFILE"; samples=result)
count

# jldopen("$PREPROCESSED/samples.jld2")

# size(mfcc(result, 16000))


# size(b)

In [ ]:
v = jldopen(TRAINFILE, "r") do f
    f["samples"]
end


v[:, :, 1, 1, 1]



